In [41]:
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI
from langchain_core.tools import Tool
from crewai_tools import tool, BaseTool,FileReadTool
from tempfile import TemporaryDirectory

from langchain_community.agent_toolkits import FileManagementToolkit
from langchain_community.utilities import GoogleSerperAPIWrapper

from langchain_google_community import GoogleSearchAPIWrapper

import os
import requests

In [42]:
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
#os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'

os.environ["GOOGLE_CSE_ID"] = ""
os.environ["GOOGLE_API_KEY"] = ""


os.environ["SERPER_API_KEY"] = ""

In [43]:
llm = ChatOpenAI(
    model = "crewai-llama2",
    base_url = "http://localhost:11434/v1")

In [44]:
# Tools

human_tool = load_tools(["human"])

# Google Search

search = GoogleSerperAPIWrapper()


# google_search_tool = Tool(
#     name = "google_search",
#     description = "Used to search information. Enter a query: ",
#     func = search.run,
# )

#search = GoogleSearchAPIWrapper()


google_search_tool = Tool(
    name="google_search",
    description="Search Google for recent results. Action Input: {{query: ""}}",
    func=search.run,
)

# google_images_tool = Tool(
#     name = "google_search_images",
#     description = "Search Google for images needed for {topic}. ",
#     func = search_images.run,
# )

# Website Requests tool
request_tool = load_tools(["requests_get"], allow_dangerous_tools = True)

# Image Creation Tool
image_tool = load_tools(["dalle-image-generator"])

@tool("image_download_tool")
def image_download_tool(url: str, file_name: str) -> str:
    """Used to download images from a url to a file. Give the url and the name the file should download as with the file ending."""

    response = requests.get(url)
    with open(f"{file_name}", "wb") as file:
    # Write the content of the response (image) to the file
        file.write(response.content)

    return f"File downloaded as {file_name}"

# File System Tools
# We'll make a temporary directory to avoid clutter
#working_directory = TemporaryDirectory()

# file_tool = FileManagementToolkit(
#     selected_tools=["read_file"],
# ).get_tools()
# file_tool

# toolkit = FileManagementToolkit(
#     #root_dir=str(working_directory.name)
# )  # If you don't provide a root_dir, operations will default to the current working directory
# toolkit.get_tools()

# file_tools = FileManagementToolkit(
#     root_dir=str("~/home/william-chandler/Research/crewai"),
#     selected_tools=["read_file", "copy_file"],
# ).get_tools()

file_tool = [FileReadTool()]

# Works as a way to make a tool but isn't compatible with load_tools

# def image_download(image_url: str, image_name: str) -> str:
#     response = requests.get(image_url)
#     with open(f"{image_name}", "wb") as file:
#     # Write the content of the response (image) to the file
#         file.write(response.content)

#     return

# image_download_tool2 = Tool(
#     name = "Image_Download_Tool",
#     func = image_download,
#     description = "Give the url of the image and the name the image should download as with the file ending."
# )


In [45]:
# Manager Agents

planning_manager = Agent(
  name = "planning_manager",
  #llm = llm,
  role='planning_manager',
  goal = (
    'Use the file_tool to read the neccessary game mechanics and images from the researcher_task. '
    'Plan how to create {topic}. '
    'Organize the tasks that must be completed. '
    'Only delegate the task of reviewing your work to the reviewer. '
    #'Delegate these tasks to co-workers. '
    #"Use the researcher to research and plan. "
    #'Use the creative manager to oversee the artistic development of {topic}'
    "When using the Action: Delegate to co-worker always specify the coworker's role. "
    #"When using the Action: Ask question to co-worker always specify the coworker's role. "
  ),
  verbose = True,
  memory = True,
  backstory = (
    "Dedicated manager who organizes tasks and delegates them effectively. "
  ),
  tools = file_tool,
  allow_delegation = False,
)

coding_manager = Agent(
  name = "coding_manager",
  #llm = llm,
  role='coding_manager',
  goal = (
    'Use the file_tool to read the coding plan from the task_plans. '
    'Over see the coding of {topic}. '
    'Organize the tasks that must be completed. '
    'Delegate these tasks to co-workers. '
    #"Use the researcher to research and plan. "
    #'Use the creative manager to oversee the artistic development of {topic}'
    "When using the Action: Delegate to co-worker always specify the coworker's role. "
    "When using the Action: Ask question to co-worker always specify the coworker's role. "
  ),
  verbose = True,
  memory = True,
  backstory = (
    "Dedicated manager who organizes tasks and delegates them effectively. "
  ),
  tools = file_tool,
  allow_delegation = True,
)

creative_manager = Agent(
    name = "creative manager", 
    role = "creative manager",
    goal = (
        "Use the file_tool to read the images plan from the planning_task. "
        "Over see the creation the neccesary images for {topic}. "
        'Organize the tasks that must be completed. '
        'Delegate these tasks to co-workers. '  # Could specify co-workers: Researcher, artist
        "Give information on and answer all questions on how to use the images in the coding of {topic}. "
        "When using the Action: Delegate to co-worker always specify the coworker's role. "
        "When using the Action: Ask question to co-worker always specify the coworker's role. "
    ),
    verbose = True,
    memory = True,
    backstory = (
        "Skilled manager, knowledgeable about the game development process. "
    ),
    allow_delegation = True,
    tools = file_tool
)

In [46]:
# Creative Agent

researcher = Agent(
    name = "researcher",
    role = "researcher",
    goal = (
        "Informer your co-workers on the game mechanics and images needed to create {topic} with {topic_info} "
        "When using the Action: google_searh use the format Action Input: {{query: ""}}. "
        "Use the google_search_tool to learn about {topic}. "
        # "Follow tasks given to by co-workers. "
        # "Answer questions and supply information to co-workers. "
      
    ),
    verbose = True,
    memory = True,
    backstory = (
        "Extremely knowledgeable about video games, and video game development. "
        "Skilled user of google search. "
    ),
    tools = [google_search_tool],
    allow_delegation = False,
)

# Artist Agent

artist = Agent(
  name = "artist",
  #llm = llm,
  role = "artist",
  goal = (
    #'Search Google for files to download. '
    #'Download files from urls found from searching google. '
    "Generate, and download a maximum of 5 images. "
    'Generate images requested by the creative manager. '
    'Download all images and name them short descriptive names. '
    'Make sure your images are scalled properly. '                                        #Topic_Specifications
    'Always include file endings in image names.'
    'Report to the creative manager the name the image has been downloaded as. '
    'All images downloaded must be of single things. Do not ask for multiple images in your prompts. '
  ),
  verbose = False,
  memory = True,
  backstory = (
    "Skilled dalle image generator prompter. "
  ),
  tools = [image_tool[0], image_download_tool, file_tool[0]], #[image_tool[0], google_images_tool, request_tool[0], download_tool],
  allow_delegation = False
)

In [47]:
# Researcher Task
researcher_task = Task (
    description = (
        "Allways give an answer. "
        "Use google_search_tool to research what game mechanics are. "
        "Use the google_search_tool to research {topic}. "
        "Find the necessary game features and images for {topic}. "
        "Make sure these game features and images include {topic_info}. "
    ),
    expected_output = (
        "A listof the necessary game features and a seperate list of images for {topic}. "
    ),
    tools = [google_search_tool],
    agent = researcher,
    human_input = False,
    async_execution = False,
    output_file = "game_research.md"
)

In [48]:
# Planning Task
planning_task = Task(
    description = (
        f"Use the file_tool to read the neccesary game features and images from the disk with file path {researcher_task.output_file}"
        "Use the information gathered by the researcher during the research task. Do not do your own research, or delegate research. "
        "Generate instructions for the art task and coding task. "
        "Delegate the task of reviewing you work to the reviewer. "
        'Only delegate the task of reviewing your work to the reviewer. '
        'Do not do any coding or generating images. '
        "Do not finalize your answer until the reviewer confirms you have included all the neccessary images. "
        #"Research and plan the game features needed for {topic}. "
        #"Research and plan the images needed for {topic}. "
    ),
    expected_output = (
        "Detailed plans and instructions for the art task and coding task. "
    ),
    tools = file_tool,
    agent = planning_manager,
    human_input = False,
    async_execution = False,
    output_file = "task_plans.md"
)

In [49]:
# Art Task
art_task = Task(
    description = (
        f"Use the file_tool to read the task instructions from the disk with file path {planning_task.output_file}. "
        #f"Do not use the download_tool to read the task instructions from the disk with file path {planning_task.output_file}."
        #"Plan the artistic development of {topic}, and delegate jobs to co-workers. "
        f"Generate and download images specified in {planning_task.output_file}. "
        #"Generate and download images to be used in {topic}. "
        "Provide information on how to use the images in {topic}. "
        "Delegate the task of reviewing you work to the reviewer. "
        "Do not finalize your answer until the reviewer confirms you have included all the neccessary images. "
    ),
    expected_output = (
        "Downloaded images for the creation of {topic}. "
        "Information on how to use the images in the creation of {topic}. "
    ),
    tools = [image_tool[0], image_download_tool],
    agent = creative_manager,
    human_input = False,
    async_execution = False,
    output_file = "art_information.md",
    )

In [50]:
# Coding Agents
programmer = Agent(
  name = "programmer",
  #llm = llm,
  role='programmer',
  goal= (
    f'Use the file_tool to read the image names from the disk with file path {art_task.output_file}. '
    'Your finale answer must be a single python script of {topic}'
    'Write perfect python code. '
    'Resolve any bugs or errors in the code. '
    f'Make sure that the images are named the same as in {art_task.output_file}. '
    'Make sure your code includes {topic_info}'
    'Make sure your code scalles images properly'                                                               #Specifications
    "When using the Action: Ask question to co-worker always specify the coworker: round_one_tester. "
    "When using the Action: Delegate to co-worker always specify the coworker's role. "
  ),
  verbose=True,
  memory=True,
  backstory=(
    "Experienced in coding and debugging.  "
  ),
  tools = file_tool,
  allow_delegation = False
)

# Creating a writer agent with custom tools and delegation capability
code_tester = Agent(
  name = "code_tester",
  #llm = llm,
  role='code_tester',
  goal= (
      "Assist the programmer in developing code. "
      "Test code to find and report bugs and errors. "
      "Make sure that the images are correctly named in the code. "

    # 'Give feedback on {topic}, and ensure that the round_one_programmer knows of any buggs or errors. '
    # 'Work with round_one_bug_fixer. '
    # 'Work before the round two and round three co-workers. '
  ),
  verbose=False,
  memory=True,
  backstory=(
    "Experienced code tester. "
  ),
  tools= file_tool,
  allow_delegation = True
)

game_tester = Agent(
  name = "game_tester",
  #llm = llm,
  role='game_tester',
  goal= (
      "Assist the programmer in developing {topic}. "
      "Make sure the programmer programms code to include {topic_info}"
      f"Make sure the programmer has made code for all the game features listed in {planning_task.output_file}. "
      f"Read the plan and tasks from the disk with file path {planning_task.output_file}"

    # 'Give feedback on {topic}, and ensure that the round_one_programmer knows of any buggs or errors. '
    # 'Work with round_one_bug_fixer. '
    # 'Work before the round two and round three co-workers. '
  ),
  verbose=False,
  memory=True,
  backstory=(
    "Experienced game tester. "
  ),
  tools= file_tool,
  allow_delegation = False
)



In [51]:
#Human Feedbacker Agent

human_feedbacker = Agent(
    name = "human_feedbacker",
    role = "human_feedbacker",
    goal = (
        "Ask the human for feedback on the creation of {topic}. "
        "Create a report of the human feedback. "
    ),
    backstory = (
        "Expirenced at game development. "
    ),
    verbose = True,
    memory = True,
    tools = human_tool,
    allow_delegation = False
)

In [52]:
# Other Agents

feedback_coder = Agent(
    name = "Feedback_Implementer",
    role = "Feedback_Implementer",
    goal =(
        'You write perfect python code. '
        'Read and use the code written by the programmer in the file game_code. '
        'Update the code written by the programmer. '
        #'You compile code and updates to that code into a final code file.'
    ),
    verbose = True,
    memory = True,
    backstory = (
        "You always make sure that all bugs and errors are fixed. "
    ),
    tools = file_tool,
    allow_delegation = True
)


# downloader = Agent(
#     name = "Downloader",
#     role = "Downloader",
#     goal = (
#         'Search Google for files to download. '
#         'Download files from urls found from searching google. '
#         'Download images created by the Artist to be used by the programmer. '
#     ),
#     verbose = True,
#     memory = True,
#     backstory = (
#         "You love downloading files and naming them proper names. "
#     ),
#     tools = [google_tool, download_tool],
#     allow_delegation = False
# )

In [53]:
# Manager Task

manager_task = Task(
  description = (
    #"Understand how to create {topic}. "
    "Plan the creation of {topic}, and delegate jobs to co-workers. "
  ),
  expected_output = (
    "A python program that runs {topic}. "
    #"Do not finalize your answer until the human says the game is complete. "
    #"Ask the human for feed back on the game before finalizing your answer. "                                                              #HUMAN FEEDBACK
    # "Do not finalize your answer until the reviewer recieves confirmation from the human that the game is complete. "                     #HUMAN FEEDBACK
    #"Do not finalize your answer until the reviewer confirms the code is ready. "                                                           
  ),
  agent = planning_manager,
  human_input = False,
  async_execution = False,
  output_file = "game.py",
)

In [54]:
# Reviewer Agent

reviewer = Agent(
    name = "Reviwer",
    role = 'Reviewer',
    goal = (
        f"Use the file_tool to read the neccessary game features and images from the disk with file path {researcher_task.output_file}. "
        "Make sure each co-worker has included all the neccessary game features and images. "
        "Answer co-workers questions on whether they have used all the neccessary game features and images. "
    ),
    verbose = True,
    memory = True,
    backstory = (
        "Knowledgeable about game design. "
    ),
    tools = file_tool,
    allow_delegation = True
)

In [55]:
# Reviewer Task
planning_review_task = Task(
    description = (
        f"Use the file_tool to read the neccessary game features and images from the disk with file path {researcher_task.output_file}."
        f"Use the file_tool to read the neccessary game features and images from the disk with file path {planning_task.output_file}."
        f"Make sure that {planning_task.output_file} contains plans to include all the neccessary game features and images listed in {researcher_task.output_file}. "
        f"Delegate the task of adding any game features and images to the to {planning_task.output_file} if they are missing. "
    ),
    expected_output = (
        "Detailed plans and instructions for the art task and coding task."
    ),
    tools = file_tool,
    agent = reviewer,
    human_input = False,
    async_execution = False,
    output_file = f"{planning_task.output_file}",
)

In [56]:
# Coding Task
coding_task = Task (
    description = (
        f"Use the file tool to read the task instructions from the disk with file path {planning_task.output_file}. "
        f"use the file tool to read the information the disk with the file path {art_task.output_file} and use these names when naming images in the code. "
        "Create perfect python code to implement everything in the coding task. "
        "Make sure that the images are correctly named in the code. "
        "Make sure that the images are scalled correctly. "
        'Make sure the code includes {topic_info}'
        "Iteratively code, test for bugs and errors, and update that code. "
        "Your final answer should include python code for {topic} and only python code for {topic}. "
        #"Delegate the task of reviewing you work to the reviewer. "
        #"Do not finalize your answer until the reviewer confirms you have included all the neccessary images. "
        #"Add features to the code to create a fully codded {topic}. "
        #"Ask the creative manager for assistance using the images if neccesary. "
    ),
    expected_output = (
        "A single executable file that contains python code for {topic}. "
    ),
    agent = coding_manager,
    human_input = False,
    async_execution = False,
    output_file = "game.py"
)

In [57]:
# Human Feedback Task
human_feedback_task = Task(
    description = (
        "Ask the human for feedback on the code for {topic}. "
        "Make a report on the human feedback to be given to the manager. "
    ),
    expected_output = (
        "A report on the human feedback. "
    ),
    agent = human_feedbacker,
    human_input = True,
    asyn_exectuion = False,
    output_file = "human_input.md"
)

In [58]:
human_feedback_planning_task = Task(
    description = (
        f"Read the human feedback from {human_feedback_task.output_file}. "
        "Generate instructions for the art task and coding task to complete the human feedback. "
        "Research and plan the game features needed for the completion of the human feedback. "
        "Research and plan the images needed for the completion of the human feedback. "
    ),
    expected_output = (
        "Detailed plans and instructions for the art task and coding task. "
    ),
    tools = [google_search_tool],
    agent = planning_manager,
    human_input = False,
    async_execution = False,
    output_file = "task_plans.md"
)

In [59]:
manager_crew = Crew(
  #agents = [manager, creative_manager, researcher, artist, programmer, code_tester, game_tester],
  #tasks = [planning_task, art_task, coding_task, human_feedback_task, human_feedback_planning_task, art_task, coding_task],
  agents = [reviewer, planning_manager, creative_manager, coding_manager, researcher, artist, programmer, code_tester, game_tester],
  tasks = [researcher_task, planning_task, art_task, coding_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  #manager_llm = ChatOpenAI(model = 'gpt-3.5-turbo'),
  verbose = 2,
  memory=True,
  cache=True,
  max_rpm=100,
  share_crew=True,
  step_callback = print("STEP ################################################"),
  max_iter = 15
)

2024-06-11 18:50:06,656 - 134438274922304 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


STEP ################################################


In [60]:
if os.path.exists('game.py'):
    os.remove("game.py")
if os.path.exists('game_research.md'):
    os.remove("game_research.md")
if os.path.exists('task_plans.md'):
    os.remove("task_plans.md")
if os.path.exists('art_information.md'):
    os.remove("art_information.md")
# Get the current directory
current_directory = os.getcwd()

# Loop through all files in the current directory
for filename in os.listdir(current_directory):
    # Check if the file ends with .png
    if filename.endswith(".png"):
        # Create the full file path
        file_path = os.path.join(current_directory, filename)
        # Delete the file
        os.remove(file_path)

    
result = manager_crew.kickoff(inputs={'topic': 'A 2d platformer game', 'topic_info': 'A dinosaur character, 5 images scalled properly, controls to make the character move left, right and jump and platforms for the character to jump on.'})

print(result)
    

 [DEBUG]: == Working Agent: researcher
 [INFO]: == Starting Task: Allways give an answer. Use google_search_tool to research what game mechanics are. Use the google_search_tool to research A 2d platformer game. Find the necessary game features and images for A 2d platformer game. Make sure these game features and images include A dinosaur character, 5 images scalled properly, controls to make the character move left, right and jump and platforms for the character to jump on.. 


> Entering new CrewAgentExecutor chain...
I need to use the google_search_tool effectively to gather information on game mechanics for a 2d platformer game with a dinosaur character. I also need to find images that meet the criteria specified.

Action: google_search
Action Input: {query: "game mechanics for 2d platformer game"} 

No good Google Search Result was found

Thought: Since the initial search did not yield relevant results, I need to refine my search query to find information on game mechanics for a 2